In [1]:
import tensorflow as tf

# 1. Cài đặt tf.distribute để chạy song song trên GPU
strategy = tf.distribute.MirroredStrategy()  # Sử dụng tất cả các GPU có sẵn

# Đường dẫn đến file CSV và thiết lập batch size
file_path = './data.csv'
batch_size = 64

# 2. Xây dựng danh sách các cột features với các lag
# Ta muốn mỗi time step chứa 3 features: body, shadow_top, shadow_bottom.
lags = list(range(15))
def get_columns_for_lag(lag):
    return [f'body_{lag}', f'shadow_top_{lag}', f'shadow_bottom_{lag}']

columns = []
for lag in lags:
    columns.extend(get_columns_for_lag(lag))
target_column = 'target'
all_columns = columns + [target_column]

# 3. Tạo dataset streaming từ CSV (sử dụng CPU)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

def parse_csv(batch):
    # Xây dựng dữ liệu với shape: (batch_size, time_steps=15, features=3)
    X_steps = []
    for lag in lags:
        # Lấy các giá trị của các feature tại mỗi lag
        step = tf.stack([
            batch[f'body_{lag}'],
            batch[f'shadow_top_{lag}'],
            batch[f'shadow_bottom_{lag}']
        ], axis=1)  # shape: (batch_size, 3)
        X_steps.append(step)
    X = tf.stack(X_steps, axis=1)  # shape: (batch_size, 15, 3)
    y = batch[target_column]
    return X, y

dataset = raw_dataset.map(parse_csv)
dataset = dataset.unbatch().enumerate()

def is_train(idx, data):
    return tf.math.mod(idx, 5) < 4

def is_val(idx, data):
    return tf.math.mod(idx, 5) == 4

train_dataset = dataset.filter(is_train).map(lambda idx, data: data)
val_dataset   = dataset.filter(is_val).map(lambda idx, data: data)

train_dataset = train_dataset.batch(batch_size).repeat()
val_dataset   = val_dataset.batch(batch_size).repeat()

# Tính số bước dựa trên số dòng (nếu cần)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 4. Xây dựng mô hình Transformer trong phạm vi của strategy
with strategy.scope():
    # Tham số mô hình
    time_steps = len(lags)       # 15
    num_features = 3             # mỗi time step có 3 features
    embedding_dim = 64           # chiều embedding cho mỗi time step
    num_transformer_blocks = 2   # số Transformer Encoder block
    head_size = 64
    num_heads = 4
    ff_dim = 128
    dropout_rate = 0.1

    # Lớp Positional Embedding: thêm thông tin vị trí cho các time step
    class PositionalEmbedding(tf.keras.layers.Layer):
        def __init__(self, sequence_length, output_dim):
            super(PositionalEmbedding, self).__init__()
            self.token_proj = tf.keras.layers.Dense(output_dim)
            self.position_embedding = tf.keras.layers.Embedding(
                input_dim=sequence_length, output_dim=output_dim
            )
            self.sequence_length = sequence_length

        def call(self, inputs):
            # inputs có shape: (batch_size, sequence_length, num_features)
            positions = tf.range(start=0, limit=self.sequence_length, delta=1)
            pos_embed = self.position_embedding(positions)  # shape: (sequence_length, output_dim)
            token_embed = self.token_proj(inputs)           # shape: (batch_size, sequence_length, output_dim)
            return token_embed + pos_embed

    # Định nghĩa Transformer Encoder block
    def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):
        # Multi-Head Self-Attention với Layer Normalization và Skip Connection
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        attn_output = tf.keras.layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(x, x)
        x = tf.keras.layers.Add()([attn_output, inputs])
        
        # Feed Forward Network với Skip Connection
        x_ff = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x_ff = tf.keras.layers.Dense(ff_dim, activation="relu")(x_ff)
        x_ff = tf.keras.layers.Dropout(dropout)(x_ff)
        x_ff = tf.keras.layers.Dense(inputs.shape[-1])(x_ff)
        return tf.keras.layers.Add()([x, x_ff])

    # Xây dựng mô hình
    inputs = tf.keras.layers.Input(shape=(time_steps, num_features))
    # Áp dụng Positional Embedding
    x = PositionalEmbedding(sequence_length=time_steps, output_dim=embedding_dim)(inputs)
    
    # Các Transformer Encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout_rate)
    
    # Global pooling qua time steps và kết nối Fully Connected layers
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_precision',
    patience=3,
    restore_best_weights=True
)

# 5. Huấn luyện mô hình (dữ liệu xử lý trên CPU, mô hình chạy trên GPU)
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 6. Lưu mô hình sau training
model.save('./predict_model_XAUUSD_transformer_v2.h5')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

Epoch 1/50
 15306/268638 ━━━━━━━━━━━━━━━━━━━━ 56:50 13ms/step - accuracy: 0.5560 - loss: 0.6855 - precision: 0.5599

KeyboardInterrupt: 